In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [2]:
import matplotlib.pyplot as plt

In [3]:
import glob

data_dir = 'datasets/weather_images/'
image_paths = glob.glob(data_dir + '*.jpg')

In [4]:
labels = ['cloudy', 'rain', 'shine', 'sunrise']

In [5]:
training_image_paths = [glob.glob(data_dir + 'train/' + l + '/*.jpg') for l in labels]
test_image_paths = [glob.glob(data_dir + 'test/' + l + '/*.jpg') for l in labels]
validation_image_paths = [glob.glob(data_dir + 'validation/' + l + '/*.jpg') for l in labels]

In [6]:
# https://stackoverflow.com/questions/952914/how-to-make-a-flat-list-out-of-list-of-lists
def flatten(list):
    return [item for sublist in list for item in sublist]

In [7]:
training_image_paths_flat = flatten(training_image_paths)
test_image_paths_flat = flatten(test_image_paths)
validation_image_paths_flat = flatten(validation_image_paths)

In [8]:
def create_labels(labels, dataset_paths):
    output_labels = []
    for x in range(len(labels)):
        for y in dataset_paths[x]:
            output_labels.append(x)
    return output_labels

In [9]:
training_labels = create_labels(labels, training_image_paths)
test_labels = create_labels(labels, test_image_paths)
validation_labels = create_labels(labels, validation_image_paths)

In [10]:
##############################################
####################STOP######################
##############################################

In [11]:
##############################################
####################GO########################
##############################################

In [12]:
# That's the Magic 
# Cast data
training_imgs = np.array([np.array(tf.keras.preprocessing.image.load_img(x, color_mode='rgb',
    target_size=(64, 64)), np.float32) for x in training_image_paths_flat])

In [13]:
# Labels
training_img_labels = np.array(training_labels)

In [14]:
import random

In [15]:
def get_one_hot(targets, num_classes):
    res = np.eye(num_classes)[np.array(targets).reshape(-1)]
    return res.reshape(list(targets.shape)+[num_classes])

In [16]:
x_train, y_train = training_imgs, training_img_labels

In [17]:
y_train = get_one_hot(y_train, 4)

In [18]:
zipped_training = list(zip(x_train, y_train))

In [19]:
random.shuffle(zipped_training)

In [20]:
x_train, y_train = list(zip(*zipped_training))

In [21]:
x_train = np.array(x_train)
y_train = np.array(y_train)

In [22]:
x_train.shape

(868, 64, 64, 3)

In [23]:
y_train.shape

(868, 4)

In [24]:
# Testing images

In [25]:
test_imgs = np.array([np.array(tf.keras.preprocessing.image.load_img(x, color_mode='rgb',
    target_size=(64, 64)), np.float32) for x in test_image_paths_flat])

In [26]:
test_img_labels = np.array(test_labels)

In [27]:
x_test, y_test = test_imgs, test_img_labels

In [28]:
y_test = get_one_hot(y_test, 4)

In [29]:
from __future__ import print_function
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Activation
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAvgPool2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.callbacks import ModelCheckpoint

from tensorflow.keras.models import model_from_json
from tensorflow.keras import backend as K

In [30]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4, activation='softmax'))

In [31]:
model.compile(
    loss='categorical_crossentropy',
    optimizer='Adam',
    metrics=['accuravacy']
)

In [32]:
epochs = 50
batch_size = 128

# Fit the model weights.
model.fit(
    x_train,
    y_train,
    epochs=epochs,
    batch_size=batch_size,
    verbose=1, validation_data=(x_test, y_test)
)
          

Train on 868 samples, validate on 153 samples
Epoch 1/50
868/868 [==============================] - 12s 14ms/sample - loss: 261.6363 - accuracy: 0.3525 - val_loss: 31.6422 - val_accuracy: 0.4837
Epoch 2/50
868/868 [==============================] - 10s 11ms/sample - loss: 12.1929 - accuracy: 0.6613 - val_loss: 1.6783 - val_accuracy: 0.7451
Epoch 3/50
868/868 [==============================] - 9s 11ms/sample - loss: 1.9703 - accuracy: 0.6924 - val_loss: 0.9314 - val_accuracy: 0.6405
Epoch 4/50
868/868 [==============================] - 9s 11ms/sample - loss: 0.9493 - accuracy: 0.7154 - val_loss: 0.8413 - val_accuracy: 0.7320
Epoch 5/50
868/868 [==============================] - 10s 11ms/sample - loss: 0.7451 - accuracy: 0.7120 - val_loss: 0.7659 - val_accuracy: 0.7255
Epoch 6/50
868/868 [==============================] - 9s 11ms/sample - loss: 0.6599 - accuracy: 0.7408 - val_loss: 0.8673 - val_accuracy: 0.6209
Epoch 7/50
868/868 [==============================] - 10s 11ms/sample - loss:

In [33]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              multiple                  896       
_________________________________________________________________
conv2d_1 (Conv2D)            multiple                  18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) multiple                  0         
_________________________________________________________________
dropout (Dropout)            multiple                  0         
_________________________________________________________________
flatten (Flatten)            multiple                  0         
_________________________________________________________________
dense (Dense)                multiple                  7372928   
_________________________________________________________________
dropout_1 (Dropout)          multiple                  0

In [34]:
model.evaluate(x_test, y_test, verbose=2)

153/1 - 0s - loss: 0.5250 - accuracy: 0.9085


[0.7441960617997287, 0.90849674]

In [49]:
model2 = Sequential()
model2.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))
model2.add(Conv2D(64, (3, 3), activation='relu'))
model2.add(MaxPooling2D(pool_size=(2, 2)))
model2.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))
model2.add(Conv2D(64, (3, 3), activation='relu'))
model2.add(MaxPooling2D(pool_size=(2, 2)))
model2.add(Dropout(0.25))
model2.add(Flatten())

model2.add(Dropout(0.5))
model2.add(Flatten())
model2.add(Dense(4, activation='softmax'))

In [50]:
model2.compile(
    loss='categorical_crossentropy',
    optimizer='Adam',
    metrics=['accuracy']
)

In [51]:
epochs = 50
batch_size = 128

# Fit the model weights.
model2.fit(
    x_train,
    y_train,
    epochs=epochs,
    batch_size=batch_size,
    verbose=1, validation_data=(x_test, y_test)
)
          

Train on 868 samples, validate on 153 samples
Epoch 1/50
868/868 [==============================] - 15s 18ms/sample - loss: 17.8986 - accuracy: 0.3848 - val_loss: 1.3465 - val_accuracy: 0.6078
Epoch 2/50
868/868 [==============================] - 13s 15ms/sample - loss: 2.1675 - accuracy: 0.6221 - val_loss: 1.0325 - val_accuracy: 0.6340
Epoch 3/50
868/868 [==============================] - 12s 14ms/sample - loss: 1.1572 - accuracy: 0.6555 - val_loss: 0.7108 - val_accuracy: 0.7516
Epoch 4/50
868/868 [==============================] - 12s 14ms/sample - loss: 0.7398 - accuracy: 0.7465 - val_loss: 0.5855 - val_accuracy: 0.8170
Epoch 5/50
868/868 [==============================] - 12s 14ms/sample - loss: 0.5293 - accuracy: 0.7995 - val_loss: 0.5597 - val_accuracy: 0.8431
Epoch 6/50
868/868 [==============================] - 12s 14ms/sample - loss: 0.5702 - accuracy: 0.8076 - val_loss: 0.6081 - val_accuracy: 0.7778
Epoch 7/50
868/868 [==============================] - 12s 13ms/sample - loss:

In [52]:
model2.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            multiple                  896       
_________________________________________________________________
conv2d_7 (Conv2D)            multiple                  18496     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 multiple                  0         
_________________________________________________________________
conv2d_8 (Conv2D)            multiple                  18464     
_________________________________________________________________
conv2d_9 (Conv2D)            multiple                  18496     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 multiple                  0         
_________________________________________________________________
dropout_29 (Dropout)         multiple                 

In [53]:
model2.evaluate(x_test, y_test, verbose=2)

153/1 - 0s - loss: 0.3302 - accuracy: 0.9020


[0.5337015073868184, 0.9019608]